In [ ]:
!sudo apt-get install build-essential swig
!pip install auto-sklearn

In [9]:
# 런타임 초기화 후 실행
import numpy as np
import pandas as pd
import autosklearn.regression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [23]:
# ===DATA===

train = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/sample_submission.csv')
train.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


In [24]:
# ===UTILS===

# 1. seperate datetime
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point)-2017)
        month.append(int(month_point)-3)
        day.append(int(day_point))
    return year, month, day


# 2. Calculate NMAE
def NMAE(true, pred):
    return np.mean(np.abs(true-pred) / true)



In [25]:
year, month, day = seperate_datetime(train)

train['year'] = year
train['month'] = month
train['day'] = day

X = train.drop(['date_time', 'number_of_rentals'], axis=1)
y = train.number_of_rentals

In [26]:
X_combination = X.copy()

In [27]:
# ===FEATURE ENGINEERING===

# 요일 정보 추가
week_day = pd.to_datetime(train['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_combination['week_day'] = le.transform(week_day)

In [28]:
X_combination = X_combination.drop(['wind_direction', 'Precipitation_Probability','humidity'], axis=1)
col_list = X_combination.columns
# 이중 for문을 사용하여 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_2=X_combination

In [29]:
# train-val split
# X_train, X_val, y_train, y_val = train_test_split(X_2, 
#                                                   y, 
#                                                   test_size=0.25, 
#                                                   random_state=42,
#                                                   shuffle=False)

In [30]:
# Build and fit a regressor

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    tmp_folder='/tmp/autosklearn_regression_example_tmp',
)
automl.fit(X_2, y, dataset_name='diabetes')

AutoSklearnRegressor(per_run_time_limit=30, time_left_for_this_task=120,
                     tmp_folder='/tmp/autosklearn_regression_example_tmp')

In [31]:
train_predictions = automl.predict(X_2)
print("Train NMAE:", NMAE(y, train_predictions))
# test_predictions = automl.predict(X_val)
# print("Test NMAE:", NMAE(y_val, test_predictions))

Train NMAE: 0.1712604864924631


In [32]:
# ===TEST===

year, month, day = seperate_datetime(test) #날짜 숫자로 분리 후 

#새로운 컬럼을 생성
test['year'] = year 
test['month'] = month
test['day'] = day

# date_time은 제거합니다.
test_X = test.drop('date_time', axis = 1)


# 요일 정보 추가

week_day = pd.to_datetime(test['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
test_X['week_day'] = le.transform(week_day)

# # 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
# test_X['temp_diff_info'] = test_X['high_temp'] - test_X['low_temp']

# # 2. "덥고 습한 날씨"를 알기 위한 정보
# test_X['sweat_info'] = test_X['high_temp'] * test_X['humidity'] 

# # 3. "춥고 바람부는 날씨"를 알기 위한 정보
# test_X['cold_info'] = test_X['low_temp'] * test_X['wind_speed'] 

In [33]:
test_X = test_X.drop(['wind_direction', 'Precipitation_Probability','humidity'], axis=1)


In [34]:
col_list = test_X.columns

for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        test_X[f'{col_list[i]}*{col_list[j]}'] = test_X[col_list[i]] * test_X[col_list[j]]

In [35]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = automl.predict(test_X)

# submission dataFrame 완성
sample_submission['number_of_rentals'] = test_yhat

# 제출 파일 생성
sample_submission.to_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/automl2.csv', index= False)

In [ ]:
# 1. 왜 train test split 하면 안될까?
# 2. val 셋으로 검증했을때의 점수랑 왜 이렇게 차이가날까?
# 3. 1 때문에 2를 하고 